In [0]:
%sql
USE CATALOG etl_project;
USE SCHEMA bigquery_db_cohort_db;

-- Build the Silver customer facts table

CREATE OR REPLACE TABLE silver_customer_facts AS

-- 1) Order each customer's purchases chronologically
WITH ordered AS (
    SELECT
        customer_id,
        order_id,
        order_date,
        -- Assign a sequential number to each order per customer
        ROW_NUMBER() OVER (
            PARTITION BY customer_id
            ORDER BY order_date
        ) AS order_rank
    FROM bronze_ecom_orders
),

-- 2) Aggregate first/second purchase and total order count
agg AS (
    SELECT
        customer_id,
        -- Earliest purchase date = cohort anchor
        MIN(order_date) AS first_purchase_date,
        -- Second purchase date (NULL if none exists)
        MIN(CASE WHEN order_rank = 2 THEN order_date END) AS second_purchase_date,
        -- Total number of purchases for basic customer profiling
        COUNT(*) AS total_orders
    FROM ordered
    GROUP BY customer_id
)

-- 3) Final selection with cohort month + date differences
SELECT
    customer_id,
    first_purchase_date,
    -- Cohort month aligns customers based on month of first purchase
    TO_DATE(DATE_TRUNC('month', first_purchase_date)) AS cohort_month,
    second_purchase_date,
    -- Days between 1st and 2nd purchase (NULL if no second purchase)
    DATE_DIFF(second_purchase_date, first_purchase_date) AS days_between,
    -- Month difference used for retention window buckets (0, 1, 2, ...)
    DATE_DIFF(month, first_purchase_date, second_purchase_date) AS months_between,
    total_orders
FROM agg;

